# LDA on Tweets Based On USA

In [2]:
#importing dataset
import pandas as pd
import numpy as np

reviews_datasets = pd.read_csv(r'C:\Debjit\ISB\03_Term-2\Foundational Project\Data Dump\CleanTweets\FP1_Merged_US_Sentiments.csv')
reviews_datasets = reviews_datasets.head(20000)


In [3]:
###cleaning tweets via function
import re
def clean_tweet(clean_txt):
    text = re.sub("RT @[\w]*:","",clean_txt)
    text = re.sub("@[\w]*","",clean_txt)
    text = re.sub("https?://[A-Za-z0-9./]*","",clean_txt)
    text = re.sub("\n","",clean_txt)
    return clean_txt


In [4]:
#cleaned tweets
reviews_datasets['clean_txt'] = reviews_datasets['clean_txt'].apply(lambda x: clean_tweet(x))
reviews_datasets.head(20)

reviews_datasets['clean_tweet'] = reviews_datasets.clean_txt.apply(clean_tweet)
reviews_datasets

,id,created_at,truncated,url,location_US_State,geo_US_State,place_US_State,createDate,clean_txt,TB_polarity,TB_subjectivity,AF_polarity,TB_sentiment,clean_tweet
0,1217234682309464064,2020-01-14T23:58:58,False,[],New Hampshire,NaN,NaN,2020-01-14,A PROVEN LIAR Trump said HE D PAY LEGAL FEES ...,0.200000,0.200000,-5.0,Neutral,A PROVEN LIAR Trump said HE D PAY LEGAL FEES ...
1,1217197393948692482,2020-01-14T21:30:47,False,[],Florida,NaN,NaN,2020-01-14,2 things are often true there is enough evide...,0.450000,0.716667,5.0,Positive,2 things are often true there is enough evide...
2,1217197457270104066,2020-01-14T21:31:02,True,[https://t.co/5R55ma3Efz],New Jersey,NaN,NaN,2020-01-14,Whether you are a Democrat or Republican Trump...,0.000000,0.000000,1.0,Neutral,Whether you are a Democrat or Republican Trump...
3,1217197459857866753,2020-01-14T21:31:03,False,[],Virginia,NaN,NaN,2020-01-14,pubs You don t say Trump Adviser Caught on Ta...,0.000000,0.000000,-2.0,Neutral,pubs You don t say Trump Adviser Caught on Ta...
4,1217197467441213441,2020-01-14T21:31:05,False,[],Texas,NaN,NaN,2020-01-14,So the shooting in Pensacola WAS a terrorist ...,-0.800000,1.000000,-3.0,Strongly Negative,So the shooting in Pensacola WAS a terrorist ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1323233360060317698,2020-11-02T11:59:52,False,[],Delaware,NaN,NaN,2020-11-02,The only economic agrument the haters have fou...,0.050000,0.400000,-3.0,Neutral,The only economic agrument the haters have fou...
19996,1323233426070216705,2020-11-02T12:00:07,False,[],Massachusetts,NaN,NaN,2020-11-02,Im proud to be a hispanic American i studied ...,0.177083,0.435417,1.0,Neutral,Im proud to be a hispanic American i studied ...
19997,1323233440980856834,2020-11-02T12:00:11,False,[],United States of America,NaN,NaN,2020-11-02,Prediction after loses next week he will quit...,-0.150000,0.050000,3.0,Neutral,Prediction after loses next week he will quit...
19998,1323233119793827842,2020-11-02T11:58:54,False,[],Georgia,NaN,NaN,2020-11-02,They gotta shut down his Twitter Trump,-0.155556,0.288889,0.0,Neutral,They gotta shut down his Twitter Trump


In [5]:
#creating vocabulary of all the words in tweets data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
analyzer='word',       
min_df=3,# minimum required occurences of a word 
stop_words='english',# remove stop words
lowercase=True,# convert all words to lowercase
token_pattern='[a-zA-Z0-9]{3,}',# num chars > 3
max_features=5000,# max number of unique words
                            )
data_matrix = vectorizer.fit_transform(reviews_datasets.clean_txt)
data_matrix

<20000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 134032 stored elements in Compressed Sparse Row format>

In [6]:
#LDA to create topics along with the probability distribution for each word in our vocabulary for each topic
lda_model = LatentDirichletAllocation(
n_components=10, # Number of topics
learning_method='online',
random_state=20,       
n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [2]:
#creating web based visualization for the topic from LDA
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

ModuleNotFoundError: No module named 'pyLDAvis'

In [7]:
#top 10 words in each topic
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['party', 'working', 'country', 'left', 'free', 'news', 'did', 'democrats', 'americans', 'trump']


Top 10 words for topic #1:
['votes', 'time', 'america', 'usa', 'clear', 'electionday', 'like', 'just', 'vote', 'trump']


Top 10 words for topic #2:
['won', 'world', 'miami', 'covid', 'presidential', '000', 'think', 'voters', 'know', 'trump']


Top 10 words for topic #3:
['violence', 'president', 'maga2020', 'love', 'trump2020landslide', 'china', 'loses', 'donald', 'maga', 'trump']


Top 10 words for topic #4:
['listen', 'win', 'does', 'trump2020', 'don', 'biden', 'make', 'media', 'election2020', 'trump']


Top 10 words for topic #5:
['joe', '2020', 'live', 'new', 'america', 'president', 'rally', 'biden', 'election', 'trump']


Top 10 words for topic #6:
['come', 'trump2020landslidevictory', 'say', 'voting', 'obama', 'american', 'going', 'trump', 'true', 'people']


Top 10 words for topic #7:
['post', 'nigeria', 'cartoon', 'rallies', 'lose', 'people', 'day', 's

In [8]:
#assigning topic to each Tweet
topic_values = lda_model.transform(data_matrix)
reviews_datasets['Topic'] = topic_values.argmax(axis=1)
reviews_datasets.head()

,id,created_at,truncated,url,location_US_State,geo_US_State,place_US_State,createDate,clean_txt,TB_polarity,TB_subjectivity,AF_polarity,TB_sentiment,clean_tweet,Topic
0,1217234689376837633,2020-01-14T23:58:59,False,[],NaN,NaN,NaN,2020-01-14,Justin Trudeau blames escalation between Trum...,0.00,0.000000,-6.0,Neutral,Justin Trudeau blames escalation between Trum...,7
1,1217196269912772608,2020-01-14T21:26:19,False,[],NaN,NaN,NaN,2020-01-14,Mini Mike Bloomberg is self made no 400 mil i...,-0.35,0.800000,2.0,Negative,Mini Mike Bloomberg is self made no 400 mil i...,4
2,1217196265295024128,2020-01-14T21:26:18,True,[https://t.co/YK6nz8VuFV],NaN,NaN,NaN,2020-01-14,Absolutely Now when it comes to selling arms ...,0.10,0.500000,3.0,Neutral,Absolutely Now when it comes to selling arms ...,9
3,1217196230926897153,2020-01-14T21:26:10,False,[],NaN,NaN,NaN,2020-01-14,pubs You don t say Trump Adviser Caught on Ta...,0.00,0.000000,-2.0,Neutral,pubs You don t say Trump Adviser Caught on Ta...,9
4,1217196220579565569,2020-01-14T21:26:08,False,[],NaN,NaN,NaN,2020-01-14,2 things are often true there is enough evide...,0.45,0.716667,5.0,Positive,2 things are often true there is enough evide...,6
